# 🧩 Crossword Solver - Tool per Cruciverba

**Strumento avanzato per la ricerca di parole con pattern e wildcard intelligenti**

Questo notebook implementa un sistema completo per:
- 🔍 Ricerca pattern con wildcard (`*`, `@`, `#`)
- 📚 Gestione multi-dizionario automatica
- 🔤 Ricerca sottostringhe con filtri di lunghezza
- 🎯 Supporto per creazione e risoluzione cruciverba

📖 **Per la documentazione completa**, consulta il file `README.md`

## 📁 Funzioni di Caricamento Dizionari

Le seguenti funzioni gestiscono automaticamente il caricamento dei file dizionario dalla cartella `dizionari/`

In [1]:
import os
import glob

def carica_tutti_dizionari(cartella_dizionari="dizionari"):
    """
    Carica automaticamente tutti i file .txt dalla cartella dizionari.
    
    Args:
        cartella_dizionari: nome della cartella contenente i dizionari (default: "dizionari")
    
    Returns:
        Dizionario con nome_file -> lista di parole
    """
    dizionari = {}
    
    # Se la cartella non esiste, cerca i file .txt nella cartella corrente
    if not os.path.exists(cartella_dizionari):
        print(f"Cartella '{cartella_dizionari}' non trovata, cerco file .txt nella cartella corrente...")
        pattern_file = "*.txt"
        base_path = ""
    else:
        pattern_file = os.path.join(cartella_dizionari, "*.txt")
        base_path = cartella_dizionari + "/"
    
    # Trova tutti i file .txt
    file_dizionari = glob.glob(pattern_file)
    
    if not file_dizionari:
        print(f"Nessun file .txt trovato in '{cartella_dizionari if os.path.exists(cartella_dizionari) else 'cartella corrente'}'")
        return dizionari
    
    print(f"Trovati {len(file_dizionari)} file dizionario:")
    
    for percorso_file in file_dizionari:
        nome_file = os.path.basename(percorso_file).replace('.txt', '')
        try:
            with open(percorso_file, 'r', encoding='utf-8') as f:
                parole = [list(riga.strip().lower()) for riga in f if riga.strip()]
                # Rimuovi duplicati
                parole_uniche = []
                parole_viste = set()
                for parola in parole:
                    parola_str = ''.join(parola)
                    if parola_str not in parole_viste:
                        parole_uniche.append(parola)
                        parole_viste.add(parola_str)
                
                dizionari[nome_file] = parole_uniche
                print(f"  - {nome_file}: {len(parole_uniche)} parole")
        
        except Exception as e:
            print(f"  - Errore caricando {nome_file}: {e}")
    
    return dizionari

def importa_dizionario(percorsi_file):
    """
    Funzione legacy per compatibilità con il codice esistente.
    """
    # Se è una stringa, convertila in lista
    if isinstance(percorsi_file, str):
        percorsi_file = [percorsi_file]
    
    parole = []
    for percorso_file in percorsi_file:
        try:
            with open(percorso_file, 'r', encoding='utf-8') as f:
                parole_file = [list(riga.strip().lower()) for riga in f if riga.strip()]
                parole.extend(parole_file)
                print(f"Importate {len(parole_file)} parole da {percorso_file}")
        except FileNotFoundError:
            print(f"Attenzione: File {percorso_file} non trovato")
        except Exception as e:
            print(f"Errore durante l'importazione di {percorso_file}: {e}")
    
    # Rimuovi duplicati mantenendo l'ordine
    parole_uniche = []
    parole_viste = set()
    for parola in parole:
        parola_str = ''.join(parola)
        if parola_str not in parole_viste:
            parole_uniche.append(parola)
            parole_viste.add(parola_str)
    
    print(f"Totale parole uniche importate: {len(parole_uniche)}")
    return parole_uniche

## 🔍 Funzioni di Ricerca

Implementazione delle funzioni core per la ricerca con wildcard e filtri

In [2]:
import re

def cerca_parole(parole, pattern):
    """
    Cerca parole che corrispondono al pattern con wildcard avanzate.
    La ricerca è case insensitive.
    
    Wildcard supportate:
    - '*': qualsiasi carattere
    - '@': vocale (a, e, i, o, u)
    - '#': consonante (qualsiasi lettera che non sia vocale)
    
    Esempi:
    - '**o**': parole di 5 lettere con 'o' come terza lettera
    - 'c@sa': parole di 4 lettere che iniziano con 'c', hanno una vocale, poi 's', poi 'a'
    - '#@ro': parole di 4 lettere: consonante, vocale, 'r', 'o'
    """
    # Converte il pattern in minuscolo per confronto case insensitive
    pattern = pattern.lower()
    
    # Definisce vocali e consonanti
    vocali = set('aeiou')
    consonanti = set('bcdfghjklmnpqrstvwxyz')
    
    risultati = []
    for parola in parole:
        if len(parola) != len(pattern):
            continue
        corrisponde = True
        for p_char, pat_char in zip(parola, pattern):
            if pat_char == '*':
                # Qualsiasi carattere va bene
                continue
            elif pat_char == '@':
                # Deve essere una vocale
                if p_char not in vocali:
                    corrisponde = False
                    break
            elif pat_char == '#':
                # Deve essere una consonante
                if p_char not in consonanti:
                    corrisponde = False
                    break
            else:
                # Carattere specifico
                if p_char != pat_char:
                    corrisponde = False
                    break
        if corrisponde:
            risultati.append(''.join(parola))
    return risultati

def cerca_sottostringa(parole, sottostringa, filtro_lunghezza=None):
    """
    Cerca tutte le parole che contengono la sottostringa specificata,
    indipendentemente dalla lunghezza della parola.
    
    Args:
        parole: Lista di parole dove ogni parola è una lista di caratteri
        sottostringa: Stringa da cercare dentro le parole
        filtro_lunghezza: Filtro per la lunghezza delle parole (es: "=7", "<5", ">10")
    
    Returns:
        Lista di parole che contengono la sottostringa
    
    Esempi:
    - cerca_sottostringa(parole, 'tro') → trova 'metro', 'altro', 'contro', ecc.
    - cerca_sottostringa(parole, 'tro', '=5') → trova solo parole di 5 lettere con 'tro'
    - cerca_sottostringa(parole, 'zione', '>8') → trova parole con 'zione' e più di 8 lettere
    """
    sottostringa = sottostringa.lower()
    risultati = []
    
    for parola in parole:
        parola_str = ''.join(parola)
        if sottostringa in parola_str:
            # Applica filtro lunghezza se specificato
            if filtro_lunghezza:
                lunghezza = len(parola_str)
                if not verifica_lunghezza(lunghezza, filtro_lunghezza):
                    continue
            risultati.append(parola_str)
    
    return risultati

def verifica_lunghezza(lunghezza, filtro):
    """
    Verifica se la lunghezza della parola soddisfa il filtro.
    
    Args:
        lunghezza: Lunghezza della parola
        filtro: Stringa di filtro (es: "=7", "<5", ">10")
    
    Returns:
        True se la lunghezza soddisfa il filtro, False altrimenti
    """
    if not filtro:
        return True
    
    if filtro.startswith('='):
        target = int(filtro[1:])
        return lunghezza == target
    elif filtro.startswith('<'):
        target = int(filtro[1:])
        return lunghezza < target
    elif filtro.startswith('>'):
        target = int(filtro[1:])
        return lunghezza > target
    
    return True

def cerca_in_tutti_dizionari(dizionari, pattern, mostra_tutti=True, max_risultati_per_dizionario=None):
    """
    Cerca un pattern in tutti i dizionari e mostra i risultati divisi per dizionario.
    
    Args:
        dizionari: Dizionario con nome_dizionario -> lista_parole
        pattern: Pattern di ricerca con wildcard OPPURE sottostringa se usa -
        mostra_tutti: Se True mostra tutti i risultati, altrimenti limita
        max_risultati_per_dizionario: Numero massimo di risultati da mostrare per ogni dizionario (solo se mostra_tutti=False)
    
    Returns:
        Dizionario con risultati per ogni dizionario
        
    Nota: Se il pattern inizia con '-', verrà trattato come ricerca di sottostringa.
    Esempi: 
    - '-tro' cerca tutte le parole che contengono 'tro'
    - '-tro=5' cerca parole che contengono 'tro' di esattamente 5 lettere
    - '-tro<7' cerca parole che contengono 'tro' di meno di 7 lettere
    - '-tro>10' cerca parole che contengono 'tro' di più di 10 lettere
    """
    tutti_risultati = {}
    totale_complessivo = 0
    
    # Determina il tipo di ricerca
    if pattern.startswith('-'):
        tipo_ricerca = "sottostringa"
        termine_ricerca = pattern[1:]  # Rimuove il -
        
        # Estrai filtro lunghezza se presente
        filtro_lunghezza = None
        for op in ['=', '<', '>']:
            if op in termine_ricerca:
                parti = termine_ricerca.split(op)
                if len(parti) == 2:
                    termine_ricerca = parti[0]
                    try:
                        filtro_lunghezza = op + parti[1]
                    except:
                        pass
                break
        
        if filtro_lunghezza:
            print(f"🔍 Ricerca sottostringa: '{termine_ricerca}' con filtro lunghezza '{filtro_lunghezza}'")
        else:
            print(f"🔍 Ricerca sottostringa: '{termine_ricerca}' (parole che contengono questa sequenza)")
    else:
        tipo_ricerca = "pattern"
        termine_ricerca = pattern
        filtro_lunghezza = None
        print(f"🔍 Ricerca pattern: '{pattern}' (lunghezza fissa con wildcard)")
    
    print("=" * 60)
    
    for nome_dizionario, parole in dizionari.items():
        if tipo_ricerca == "sottostringa":
            risultati = cerca_sottostringa(parole, termine_ricerca, filtro_lunghezza)
        else:
            risultati = cerca_parole(parole, pattern)
            
        tutti_risultati[nome_dizionario] = risultati
        totale_complessivo += len(risultati)
        
        print(f"\n📚 {nome_dizionario.upper()}: {len(risultati)} risultati")
        if risultati:
            if mostra_tutti or max_risultati_per_dizionario is None:
                # Mostra tutti i risultati
                for i, parola in enumerate(risultati, 1):
                    print(f"  {i:3d}. {parola}")
            else:
                # Mostra solo i primi risultati
                risultati_da_mostrare = risultati[:max_risultati_per_dizionario]
                for i, parola in enumerate(risultati_da_mostrare, 1):
                    print(f"  {i:2d}. {parola}")
                
                if len(risultati) > max_risultati_per_dizionario:
                    print(f"     ... e altri {len(risultati) - max_risultati_per_dizionario} risultati")
        else:
            print("     Nessun risultato trovato")
    
    print("=" * 60)
    print(f"📊 TOTALE: {totale_complessivo} parole trovate in {len(dizionari)} dizionari")
    
    return tutti_risultati

## 🚀 Inizializzazione Sistema

Caricamento automatico di tutti i dizionari disponibili

In [3]:
# Caricamento automatico dizionari (solo se non già caricati)

if 'dizionari' not in globals() or not dizionari:
    print("🔄 Caricamento dizionari in corso...")
    dizionari = carica_tutti_dizionari()
else:
    print("✅ Dizionari già caricati!")

print(f"\n📊 STATO SISTEMA:")
print(f"   • Parole totali: {sum(len(parole) for parole in dizionari.values()):,}")
print(f"   • Dizionari: {', '.join(dizionari.keys())}")
print(f"   • Sistema pronto per la ricerca! 🎯")

🔄 Caricamento dizionari in corso...
Trovati 2 file dizionario:
  - eng: 466546 parole
  - ita: 279894 parole

📊 STATO SISTEMA:
   • Parole totali: 746,440
   • Dizionari: eng, ita
   • Sistema pronto per la ricerca! 🎯


In [4]:
# Esempi di ricerca - Pattern e Sottostringhe

# Verifica che i dizionari siano caricati
if 'dizionari' not in globals() or not dizionari:
    print("⚠️ ATTENZIONE: Esegui prima la cella di inizializzazione (cella 7)")
else:
    print("🎯 ESEMPI DI RICERCA")
    print("=" * 40)
    
    # Esempio 1: Pattern fisso
    print("\n📏 Pattern (lunghezza fissa):")
    cerca_in_tutti_dizionari(dizionari, '#@ro', mostra_tutti=False, max_risultati_per_dizionario=3)
    
    print("\n" + "-" * 40)
    
    # Esempio 2: Sottostringa
    print("\n🔍 Sottostringa (qualsiasi lunghezza):")
    cerca_in_tutti_dizionari(dizionari, '-tro', mostra_tutti=False, max_risultati_per_dizionario=3)

🎯 ESEMPI DI RICERCA

📏 Pattern (lunghezza fissa):
🔍 Ricerca pattern: '#@ro' (lunghezza fissa con wildcard)

📚 ENG: 30 risultati
   1. biro
   2. boro
   3. buro
     ... e altri 27 risultati

📚 ITA: 34 risultati
   1. baro
   2. biro
   3. boro
     ... e altri 31 risultati
📊 TOTALE: 64 parole trovate in 2 dizionari

----------------------------------------

🔍 Sottostringa (qualsiasi lunghezza):
🔍 Ricerca sottostringa: 'tro' (parole che contengono questa sequenza)

📚 ENG: 4748 risultati
   1. abiotrophy
   2. abiotrophic
   3. accutron
     ... e altri 4745 risultati

📚 ITA: 3528 risultati
   1. aberrometro
   2. accelerometro
   3. accentro
     ... e altri 3525 risultati
📊 TOTALE: 8276 parole trovate in 2 dizionari


## 💡 Guida Rapida

### ? Wildcard Principali:
- **`*`** = qualsiasi carattere  
- **`@`** = vocale (a,e,i,o,u)
- **`#`** = consonante
- **`-testo`** = sottostringa (con filtri lunghezza: `=5`, `<10`, `>8`)

### ? Esempi Veloci:
```python
cerca_in_tutti_dizionari(dizionari, 'c*sa')     # Pattern fisso
cerca_in_tutti_dizionari(dizionari, '-tro=5')   # Sottostringa con filtro
```

? **Per guida completa, esempi e troubleshooting** → consulta `README.md`

In [5]:
# 🎯 RICERCA INTERATTIVA

# Verifica che i dizionari siano caricati
if 'dizionari' not in globals() or not dizionari:
    print("⚠️ ERRORE: Dizionari non caricati!")
    print("👉 Esegui prima la cella di inizializzazione (cella 7)")
else:
    print("🔍 MODALITÀ RICERCA INTERATTIVA")
    print("━" * 50)
    print("📏 Pattern fisso: c*sa, #@ro, *@*@*")
    print("🔤 Sottostringa: -tro, -zione=10, -mente<8")
    print("━" * 50)
    
    try:
        pattern_ricerca = input("🔍 Inserisci la tua ricerca: ").strip()
        
        if pattern_ricerca:
            print(f"\n🎯 Ricerca per: '{pattern_ricerca}'")
            print("=" * 60)
            risultati_finali = cerca_in_tutti_dizionari(dizionari, pattern_ricerca)
            print(f"\n✅ Ricerca completata!")
        else:
            print("❌ Ricerca vuota, inserisci un pattern valido!")
            
    except KeyboardInterrupt:
        print("\n🛑 Ricerca interrotta dall'utente")
    except Exception as e:
        print(f"\n❌ Errore durante la ricerca: {e}")

🔍 MODALITÀ RICERCA INTERATTIVA
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📏 Pattern fisso: c*sa, #@ro, *@*@*
🔤 Sottostringa: -tro, -zione=10, -mente<8
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 Ricerca per: '-tro=5'
🔍 Ricerca sottostringa: 'tro' con filtro lunghezza '=5'

📚 ENG: 53 risultati
    1. altro
    2. atro-
    3. etrog
    4. intro
    5. jitro
    6. latro
    7. metro
    8. nitro
    9. petro
   10. retro
   11. stroh
   12. stroy
   13. strom
   14. strop
   15. strow
   16. troad
   17. troak
   18. troas
   19. troat
   20. troca
   21. troch
   22. trock
   23. troco
   24. trode
   25. trodi
   26. troff
   27. troft
   28. trogs
   29. troic
   30. trois
   31. troys
   32. troke
   33. troll
   34. tromp
   35. trona
   36. tronc
   37. trone
   38. tronk
   39. troop
   40. troot
   41. trooz
   42. trop-
   43. trope
   44. tropy
   45. troth
   46. trots
   47. troue
   48. troup
   49. trout
   50. trouv
   51. trove
   52. trows
   53. vitro